In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from langchain.schema import HumanMessage
from langchain.chat_models import AzureChatOpenAI
os.environ["AZURE_OPENAI_API_KEY"]  = "b82effcf491e45a088b1cd578713311c"
os.environ["OPENAI_API_VERSION"]    = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://usesharedaopenai001.openai.azure.com/'
os.environ["OPENAI_API_TYPE"]       = 'azure'
# Variables-------------------------------------------------------
index                               = "azure-cognitive-search-vector-demo"
azure_search_endpoint               = 'https://genai0.search.windows.net'
MODEL                               = "gtp35turbo-latest"
key                                 = 'lvhCA67EeE3JRyxyem5L0wGJSfOxscm2jft887ECdJAzSeDzoCNZ'
model                               = "text-embedding-ada-002"
credential                          = AzureKeyCredential(key)
COMPLETION_TOKENS                   = 1000
top_search_vector_k                 = 5

In [2]:
llm = AzureChatOpenAI(model_name=MODEL, temperature=0)

In [3]:
from langchain import PromptTemplate

## Prompt 1

In [4]:
# create the prompt, here we use multiple inputs
prompt = PromptTemplate(
    template=""""/
    You are a helpful Name Entity Recognition System. From the following text:    
    
    <<{text}>>
    
    Extract this target word class: '{word_class}'. Return the extracted element in a list. 
    For word class related to dates or years intervals always return it in the format 'YYYY-MM-DD'.
    For intervals with a start-date and end-date return both dates in a list. 
    For word class 'company_name' return it in capital letters.
    """,
    input_variables=["word_class", "text"]
)

#query = "What is the revenue of Pfizer from july 2020 to october 2021"
query = "In agreement with the information outlined in the income statement, what is the FY2015 - FY2017 3 year average net profit margin (as a %) for Best Buy? Answer in units of percents and round to one decimal place."
word_class = "dates"
#word_class = "company_name"

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    word_class = word_class,
    text = query,
    )

In [5]:
# make a prediction
#prediction = llm.predict(prompt_formatted_str)

# print the prediction
#print(prediction)

## Few-shot prompt

In [6]:
import pandas as pd
df = pd.read_csv('../../data/financebench_sample_150.csv')

In [14]:
prompt = PromptTemplate(
    template=""""/
    You are a helpful Name Entity Recognition System, specialized in extracting 'dates' from financial questions.   
    You should always return an interval where the first element represents the start-date and the second element represents the end-date. 
    If 'dates' are expressed in 'month-year', 'quarter-year', 'year-only', or any other time unit you always respond using JSON format with
    this date notation: "YYYY-MM-DD", "YYYY-MM-DD".    

    Q: What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by
        relying on the details shown in the cash flow statement.
    A: ("2018-01-01","2018-12-31")

    Q: What is Adobe's year-over-year change in unadjusted operating income from FY2015 to FY2016 
        (in units of percents and round to one decimal place)? Give a solution to the question by using the income statement.
    A: ("2015-01-01","2016-12-31")

    Q: What is the FY2017 return on assets (ROA) for Coca Cola? ROA is defined as: 
        FY2017 net income/(average total assets between FY2016 and FY2017). 
        Round your answer to two decimal places. Give a response to the question by relying on 
        the details shown in the balance sheet and the P&L statement.
    A: ("2016-01-01","2017-12-01")

    Q: What is the FY2017 - FY2019 3 year average of capex as a '%' of revenue for Activision Blizzard? 
        Answer in units of percents and round to one decimal place. Calculate (or extract) the answer from 
        the statement of income and the cash flow statement.
    A: ("2017-01-01","2019-12-31")

    Q: {query}
    A: 
    """,
    input_variables=["query"]
)

In [15]:
#query = "What is the revenue of Pfizer from july 2020 to october 2021"
query = "In agreement with the information outlined in the income statement, what is the FY2015 - FY2017 3 year average net profit margin (as a %) for Best Buy? Answer in units of percents and round to one decimal place."
#word_class = "company_name"

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(    
    query = query,
)

In [16]:
# make a prediction
prediction = llm.predict(prompt_formatted_str)

In [17]:
print(prediction.strip("'"))

("2015-01-01","2017-12-31")


In [18]:
from tenacity import retry, wait_random_exponential, stop_after_attempt
import time

# Define a function for the process you want to retry
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def process_row(question):
    prompt_formatted_str = prompt.format(query=question)
    start_time = time.time()  # Start time
    response = llm.predict(prompt_formatted_str)
    end_time = time.time()  # End time
    duration = end_time - start_time
    return response, duration

intervals = []
durations = []

for index, row in df.iterrows():
    question = row['question']
    try:
        interval, duration = process_row(question)
        intervals.append(interval.strip("'"))
        durations.append(duration)
    except Exception as e:
        print(f"Error processing row: {e}")
        intervals.append(None)
        durations.append(None)

In [19]:
# Calculate mean duration
mean_duration = sum(d for d in durations if d is not None) / len([d for d in durations if d is not None])
mean_duration

0.5230564053853353

In [20]:
intervals

['("2018-01-01","2018-12-31")',
 '("2018-12-31","2018-12-31")',
 '("2022-01-01","2022-12-31")',
 '("FY2022", "FY2022")',
 '("2022-01-01","2022-12-31")',
 '("2023-04-01","2023-06-30")',
 '("2023-04-01","2023-06-30")',
 'No specific date range is mentioned in the question, so the answer does not require a date interval.',
 '("2018-01-01","2019-12-31")',
 '("2017-01-01","2019-12-31")',
 '("2015-01-01","2015-12-31")',
 '("2015-01-01","2016-12-31")',
 '("2017-01-01","2017-12-31")',
 '("2022-01-01","2022-12-31")',
 '("2022-01-01","2022-12-31")',
 '("2022-01-01","2022-12-31")',
 '("2022-01-01","2022-12-31")',
 '("2021-01-01","2022-12-31")',
 '("2016-01-01","2017-12-31")',
 '("2016-01-01","2017-12-31")',
 '("2019-01-01","2019-12-31")',
 '("2020-12-31","2020-12-31")',
 '("2022-07-01","2022-07-01")',
 '("2022-01-01","2023-12-31")',
 '("2021-01-01","2023-12-31")',
 'null',
 '("2023-01-01","2023-12-31")',
 '("2023-04-01","2023-06-30")',
 '("2023-01-01","2023-12-31")',
 '("2022-01-01","2023-12-31")